In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
unemp_county = pd.read_csv("/content/drive/My Drive/Datasets/unemployment-by-county-us/output.csv")
unemp_county.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [3]:
df = pd.read_csv("/content/drive/My Drive/Datasets/minwage.csv")
actual_min_wage = pd.DataFrame()

for name, group in df.groupby("State"):
  if actual_min_wage.empty:
    actual_min_wage = group.set_index("Year")[["Low.2018"]].rename(columns = {"Low.2018":name})
  else:
    actual_min_wage = actual_min_wage.join(group.set_index("Year")[["Low.2018"]].rename(columns = {"Low.2018":name}))

actual_min_wage.head()

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Federal (FLSA),Florida,Georgia,Guam,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Puerto Rico,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,U.S. Virgin Islands,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1968,0.0,15.12,3.37,1.12,11.88,7.20,10.08,9.00,9.00,8.28,0.0,0.00,9.00,9.00,8.28,0.00,8.28,0.0,0.0,4.68,0.0,10.08,7.20,11.52,9.00,5.04,0.0,0.0,0.00,7.20,9.00,10.08,10.08,8.28,11.52,7.20,7.20,5.40,7.20,9.00,8.28,3.10,10.08,0.0,3.06,0.0,0.00,NaN,7.20,10.08,0.0,11.52,7.20,9.00,8.64
1969,0.0,14.33,3.19,1.07,11.26,6.83,9.56,8.53,8.53,7.85,0.0,0.00,8.53,8.53,7.85,0.00,7.85,0.0,0.0,4.44,0.0,9.56,6.83,10.92,8.53,4.78,0.0,0.0,0.00,6.83,8.53,9.56,9.56,7.85,10.92,6.83,6.83,5.12,6.83,8.53,7.85,2.94,9.56,0.0,2.90,0.0,0.00,NaN,6.83,9.56,0.0,10.92,6.83,8.53,8.19
1970,0.0,13.54,3.02,7.09,10.64,6.45,10.32,8.06,10.32,8.38,0.0,0.00,10.32,10.32,8.06,0.00,8.06,0.0,0.0,4.19,0.0,10.32,8.38,10.32,8.06,4.51,0.0,0.0,0.00,6.45,8.38,9.35,9.67,8.38,10.32,8.06,6.45,4.84,6.45,8.06,8.38,2.77,10.32,0.0,6.45,0.0,0.00,NaN,6.45,10.32,0.0,10.32,6.45,8.38,8.38
1971,0.0,12.99,2.89,6.80,10.20,6.18,9.89,7.73,9.89,8.04,0.0,0.00,9.89,9.89,7.73,0.00,7.73,0.0,0.0,4.02,0.0,9.89,8.04,9.89,7.73,4.33,0.0,0.0,0.00,6.18,8.04,8.97,9.28,8.04,9.89,7.73,6.18,4.64,6.18,7.73,8.04,2.66,9.89,0.0,6.18,0.0,0.00,NaN,6.18,9.89,0.0,9.89,6.18,8.04,8.04
1972,0.0,12.57,2.80,7.19,9.88,5.99,11.08,9.58,9.58,9.58,0.0,7.49,11.38,9.58,8.38,8.38,7.49,0.0,0.0,3.89,0.0,8.38,9.58,10.48,9.58,4.49,0.0,0.0,9.58,5.99,9.58,9.58,8.98,7.78,11.08,8.68,5.99,4.49,8.38,7.49,9.58,3.89,9.58,0.0,5.99,0.0,8.38,NaN,7.19,9.58,0.0,9.58,7.19,8.68,8.98


In [5]:
import numpy as np

actual_min_wage = actual_min_wage.replace(0, np.NaN).dropna(axis=1)
actual_min_wage.head()

,Alaska,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Federal (FLSA),Guam,Hawaii,Idaho,Indiana,Kentucky,Maine,Maryland,Massachusetts,Michigan,Minnesota,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Puerto Rico,Rhode Island,South Dakota,Utah,Vermont,Washington,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1968,15.12,1.12,11.88,7.20,10.08,9.00,9.00,8.28,9.00,9.00,8.28,8.28,4.68,10.08,7.20,11.52,9.00,5.04,7.20,9.00,10.08,10.08,8.28,11.52,7.20,7.20,5.40,7.20,9.00,8.28,3.10,10.08,3.06,7.20,10.08,11.52,7.20,9.00,8.64
1969,14.33,1.07,11.26,6.83,9.56,8.53,8.53,7.85,8.53,8.53,7.85,7.85,4.44,9.56,6.83,10.92,8.53,4.78,6.83,8.53,9.56,9.56,7.85,10.92,6.83,6.83,5.12,6.83,8.53,7.85,2.94,9.56,2.90,6.83,9.56,10.92,6.83,8.53,8.19
1970,13.54,7.09,10.64,6.45,10.32,8.06,10.32,8.38,10.32,10.32,8.06,8.06,4.19,10.32,8.38,10.32,8.06,4.51,6.45,8.38,9.35,9.67,8.38,10.32,8.06,6.45,4.84,6.45,8.06,8.38,2.77,10.32,6.45,6.45,10.32,10.32,6.45,8.38,8.38
1971,12.99,6.80,10.20,6.18,9.89,7.73,9.89,8.04,9.89,9.89,7.73,7.73,4.02,9.89,8.04,9.89,7.73,4.33,6.18,8.04,8.97,9.28,8.04,9.89,7.73,6.18,4.64,6.18,7.73,8.04,2.66,9.89,6.18,6.18,9.89,9.89,6.18,8.04,8.04
1972,12.57,7.19,9.88,5.99,11.08,9.58,9.58,9.58,11.38,9.58,8.38,7.49,3.89,8.38,9.58,10.48,9.58,4.49,5.99,9.58,9.58,8.98,7.78,11.08,8.68,5.99,4.49,8.38,7.49,9.58,3.89,9.58,5.99,7.19,9.58,9.58,7.19,8.68,8.98


In [6]:
def get_min_wage(year, state):
  try:
    return actual_min_wage.loc[year][state]
  except:
    return np.NAN

get_min_wage(2012, 'Colorado')

8.33

In [8]:
%%time

unemp_county['min_wage'] = list(map(get_min_wage, unemp_county['Year'], unemp_county['State']))

CPU times: user 2min 1s, sys: 59 ms, total: 2min 1s
Wall time: 2min 1s


In [25]:
unemp_county[['Rate', 'min_wage']].corr()

,Rate,min_wage
Rate,1.000000,0.153047
min_wage,0.153047,1.000000


In [26]:
unemp_county[['Rate', 'min_wage']].cov()

,Rate,min_wage
Rate,9.687873,0.651586
min_wage,0.651586,1.874228


In [27]:
pres16 = pd.read_csv("/content/drive/My Drive/Datasets/pres16results.csv")
pres16.head()

,county,fips,cand,st,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


In [31]:
len(unemp_county)

885548

In [0]:
county_2015 = unemp_county.copy()[(unemp_county['Year']==2015) & (unemp_county['Month']=='February')]

In [29]:
pres16['st'].unique()

array(['US', 'CA', 'FL', 'TX', 'NY', 'PA', 'IL', 'OH', 'MI', 'NC', 'GA',
       'VA', 'NJ', 'MA', 'WI', 'MN', 'MO', 'WA', 'IN', 'CO', 'TN', 'MD',
       'SC', 'AL', 'AZ', 'LA', 'KY', 'OR', 'CT', 'IA', 'OK', 'MS', 'KS',
       'NV', 'AR', 'UT', 'NE', 'NM', 'ME', 'NH', 'WV', 'ID', 'MT', 'RI',
       'DE', 'HI', 'SD', 'ND', 'VT', 'DC', 'WY', 'AK', nan], dtype=object)

In [28]:
state_abbv = pd.read_csv("/content/drive/My Drive/Datasets/state_abbv.csv", index_col=0)
state_abbv = state_abbv[["Postal Code"]]
state_abbv.head()

,Postal Code
State/District,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


In [0]:
state_abbv_dict = state_abbv.to_dict()['Postal Code']

In [0]:
county_2015['State'] = county_2015['State'].map(state_abbv_dict)

In [34]:
county_2015.tail()

,Year,Month,State,County,Rate,min_wage
2797,2015,February,ME,Somerset County,8.4,7.92
2798,2015,February,ME,Oxford County,6.8,7.92
2799,2015,February,ME,Knox County,6.1,7.92
2800,2015,February,ME,Piscataquis County,7.0,7.92
2801,2015,February,ME,Aroostook County,7.2,7.92


In [35]:
pres16.columns

Index(['county', 'fips', 'cand', 'st', 'pct_report', 'votes', 'total_votes',
       'pct', 'lead'],
      dtype='object')

In [36]:
pres16.rename(columns={'county':'County', 'st':'State'}, inplace=True)
pres16.head()

,County,fips,cand,State,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


In [0]:
for df in [county_2015, pres16]:
  df.set_index(['County', 'State'], inplace=True)

In [39]:
pres16 = pres16[pres16['cand'] == 'Donald Trump']
pres16 = pres16[['pct']]
pres16.dropna(inplace=True)
pres16.head()

pct
County State          
NaN    US     0.472993
       CA     0.330641
       FL     0.490640
       TX     0.525830
       NY     0.374752

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
all_together = county_2015.merge(pres16, on=['County', 'State'])
all_together.dropna(inplace=True)

In [41]:
all_together.drop('Year', axis=1, inplace=True)
all_together.head()

,,Month,Rate,min_wage,pct
County,State,,,,
Major County,OK,February,2.6,2.11,0.864960
Pottawatomie County,OK,February,4.5,2.11,0.701342
Johnston County,OK,February,6.5,2.11,0.770057
Jefferson County,OK,February,5.0,2.11,0.812367
Beaver County,OK,February,2.8,2.11,0.888243


In [42]:
all_together.corr()

,Rate,min_wage,pct
Rate,1.000000,0.186689,-0.085985
min_wage,0.186689,1.000000,-0.325036
pct,-0.085985,-0.325036,1.000000


In [43]:
all_together.cov()

,Rate,min_wage,pct
Rate,5.743199,0.683870,-0.031771
min_wage,0.683870,2.336451,-0.076602
pct,-0.031771,-0.076602,0.023772
